# Lab 2 - Regression and contrasts

In this lab you will create a faces > houses contrast on data from Haxby et al. 2001. In that study, different categories of images were shown to subjects. The original purpose of the study was to show that activation patterns for different concepts are distributed across the ventral cortex. Since faces and houses were among the stimuli presented, we will use the data to create a contrast between them.

Run the cells below to download the data and load categories labels and brain data.

In [ ]:
# Don't change this cell; just run it. 
# The result will give you directions about how to log in to the submission system, called OK.
# Once you're logged in, you can run this cell again, but it won't ask you who you are because
# it remembers you. However, you will need to log in once per assignment.
from client.api.notebook import Notebook
ok = Notebook('lab2.ok')
_ = ok.auth(inline=True)

In [ ]:
N_RUNS = 6

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from nilearn.datasets import fetch_haxby
haxby = fetch_haxby(subjects=(1,))

In [ ]:
!gunzip -c /home/jovyan/nilearn_data/haxby2001/subj1/bold.nii.gz > /home/jovyan/nilearn_data/haxby2001/subj1/bold.nii

In [ ]:
import nibabel
from scipy.signal import detrend
img = nibabel.load(haxby['func'][0][:-3])
raw_data = img.get_data().T[:N_RUNS * 121]

mask = nibabel.load(haxby['mask']).get_data().T.astype('bool')
data = np.zeros((raw_data.shape[0], mask.sum()), dtype='float32')

# load, mask, and zscore the data iteratively (because it's a lot of data)
for raw_run, masked_run in zip(raw_data.reshape((-1, 121) + raw_data.shape[1:]),
                               data.reshape(-1, 121, mask.sum())):
    masked = raw_run[:, mask]
    masked = detrend(masked, axis=0)
    mean = masked.mean(0) # not necessary
    stdev = masked.std(0)
    masked_run[:] = (masked - mean) / (stdev + 1e-8)
del masked, mean, stdev

from nistats.hemodynamic_models import glover_hrf
hrf = glover_hrf(tr=2, oversampling=1)


In [ ]:
target = np.recfromcsv(haxby['session_target'][0], delimiter=' ')
category_labels = np.array(list(map(bytes.decode, target['labels'])))[:N_RUNS * 121]


1\. [0.5pts] Run the cells preparing and loading the data. Print the unique labels in `category_labels` to get an idea of what was presented.

In [ ]:
# 1



2\. [3pts] For each unique label in `category_labels` create the stimulus time series by computing the array indicating which of `category_labels` corresponds to the current label. Then create the response time series by convolving (use `np.convolve`) with the hrf  and cropping back to the original length of the stimulus time series. Collect the response time series in a list, convert them to a numpy array using `np.stack` to obtain a design matrix of shape `(len(category_labels), len(np.unique(category_labels)))`. Store this matrix in the name `response_design`.

In [ ]:
# 2



3\. [2pts] Using `sklearn.linear_model.LinearRegression`, compute the regression weights for `response_design` and `data`. Store them in the name `weights`.

In [ ]:
# 3


4\. [2pts] Identify which columns of `weights` correspond to `faces` and `houses`. Subtract the one corresponding to houses from that corresponding to faces. Call the output `contrast_faces_vs_houses`.

In [ ]:
# 4


5\. [2.5pts] In order to be able to visualize the contrast, you first need to unmask the voxels back into a volume. Do this using the function `unmask`. Then plot slice 27 of the first axis. You should see an axial slice. Use `vmin, vmax = -2, 2` and the colormap `RdBu_r`. In `plt.imshow` you can set `origin` to `"lower"`. Also add a colorbar using `plt.colorbar`.

In [ ]:
def unmask(masked):
    output = np.zeros_like(mask, dtype='float64')
    output[mask] = masked
    return output

In [ ]:
# 5


Make sure to submit!

In [ ]:
_ = ok.submit()